In [ ]:
try:
    %load_ext lab_black
except ModuleNotFoundError:
    print("nb_black not installed")

In [ ]:
# currently this file is broken

In [ ]:
import os
import re
import json
import csv
import datetime
import requests
from time import time
from tqdm import tqdm

from common import save_graph, load_graph, get_from_playlists_from_time_range, id_to_url
from scrape_helpers import (
    get_exported_youtube_playlist_ids,
    scrape_from_list,
    scrape_playlist,
    scrape_all_playlists,
)

seconds_in_month = 60 * 60 * 24 * 30.4

home = os.path.expanduser("~")
playlists_path = f"{home}/.yourtube/Takeout/YouTube and YouTube Music/playlists"

# Scrape all playlists

In [ ]:
scrape_all_playlists()

# Scrape neighborhood of recently liked videos


In [ ]:
G = load_graph()

In [ ]:
seconds_in_year = 60 * 60 * 24 * 365
start_time = time() - seconds_in_year * 3
sources = get_from_playlists_from_time_range(G, start_time)
print(len(sources))

In [ ]:
_, out_nodes = zip(*G.out_edges(sources))
print(len(out_nodes))
out_nodes = set(out_nodes)
print(len(out_nodes))

In [ ]:
scrape_from_list(out_nodes, G, skip_if_fresher_than=seconds_in_month)

In [ ]:
save_graph(G)